# **Classifying Dogs and Cats with a VGG-16 model**

Original dataset : [Cats-vs-Dogs : image dataset for binary classification](https://www.kaggle.com/shaunthesheep/microsoft-catsvsdogs-dataset)
*** 

# **Introduction** 

A pre-trained ConvNet is composed of two parts :
* the **convolutional base** (convolutions + poolings)
* the **classifier** (fully connected network).

We keep the convolutional base, which probably has the most generic informations.  

We change the classifier to adapt to the problem we have to deal with. 

***

In [1]:
# Libraries
import os, shutil
import numpy as np
import matplotlib.pyplot as plt
import cv2

from keras import layers
from keras import models, optimizers
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.applications.vgg16 import VGG16

***

# **Data**

* Whole dataset : around 25,000 images (around 12,500 of each class)
  
* We use just a small part of those images. 

* Steps 
  * Load the dataset
  * Create 3 sets :
    * a training set with 1000 images
    * a validation set with 500 images
    * a test set with 500 images.

* Functions we use :
  * os.mkdir() : create a new folder
  * os.path.join(path, string) : add the string to the path
  * shutil.copyfile(source, destination) : copy the file from the source to the destination.

In [2]:
# path to the folder where the whole dataset is stored
original_dataset_dir = '../input/microsoft-catsvsdogs-dataset/PetImages'

# create a folder to store our small sample of images
base_dir = '../cats_and_dogs_small'
os.mkdir(base_dir)

In [3]:
# create sub-folders for training, validation and test sets
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)

validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)

test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# create sub-folders Cat and Dog 
train_Cat_dir = os.path.join(train_dir, 'Cat')
os.mkdir(train_Cat_dir)

train_Dog_dir = os.path.join(train_dir, 'Dog')
os.mkdir(train_Dog_dir)

validation_Cat_dir = os.path.join(validation_dir, 'Cat')
os.mkdir(validation_Cat_dir)

validation_Dog_dir = os.path.join(validation_dir, 'Dog')
os.mkdir(validation_Dog_dir)

test_Cat_dir = os.path.join(test_dir, 'Cat')
os.mkdir(test_Cat_dir)

test_Dog_dir = os.path.join(test_dir, 'Dog')
os.mkdir(test_Dog_dir)

In [4]:
# copy images from the whole dataset to different folders :

# copy the first 1000 cats images to the folder train_dir 
fnames = ['{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir + '/Cat', fname)
    dst = os.path.join(train_Cat_dir, fname)
    shutil.copyfile(src, dst)
    
# copy the following 500 cats images to the folder validation_dir 
fnames = ['{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir + '/Cat', fname)
    dst = os.path.join(validation_Cat_dir, fname)
    shutil.copyfile(src, dst)

# copy the following 500 cats images to the folder test_dir    
fnames = ['{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir + '/Cat', fname)
    dst = os.path.join(test_Cat_dir, fname)
    shutil.copyfile(src, dst)

    
# copy the first 1000 dogs images to the folder train_dir 
fnames = ['{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir + '/Dog', fname)
    dst = os.path.join(train_Dog_dir, fname)
    shutil.copyfile(src, dst)
    
# copy the following 500 dogs images to the folder validation_dir 
fnames = ['{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir + '/Dog', fname)
    dst = os.path.join(validation_Dog_dir, fname)
    shutil.copyfile(src, dst)

# copy the following 500 dogs images to the folder test_dir    
fnames = ['{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir + '/Dog', fname)
    dst = os.path.join(test_Dog_dir, fname)
    shutil.copyfile(src, dst)

In [5]:
# quick check
print("total training cat images :", len(os.listdir(train_Cat_dir)))
print("total training dog images :", len(os.listdir(train_Dog_dir)))
print("total validation cat images :", len(os.listdir(validation_Cat_dir)))
print("total validation dog images :", len(os.listdir(validation_Dog_dir)))
print("total test cat images :", len(os.listdir(test_Cat_dir)))
print("total test dog images :", len(os.listdir(test_Dog_dir)))

In [6]:
# size of an image
img = cv2.imread(train_Cat_dir + '/' + os.listdir(train_Cat_dir)[0])
img.shape

In [7]:
# error in data
# We just replace 666.jpg by 665.jpg
shutil.copyfile(train_Cat_dir + '/665.jpg', train_Cat_dir + '/666.jpg')

***

# **Load a pre-trained model**

We use **VGG16()**, with arguments :
* weights : from the training on the dataset ImageNet (1.4 millions of images, 1000 classes)
* include_top : include or not the top part of the network, the fully-connected one
* input_shape.

NB : error when downloading the model 
* [forum kaggle](https://www.kaggle.com/questions-and-answers/128824#735972)
* [forum kaggle](https://www.kaggle.com/getting-started/40246)
* first : 'Add Data' >> 'vgg16' 

In [8]:
conv_base = VGG16(weights='../input/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, input_shape=(150,150,3))

In [9]:
conv_base.summary()

***

# **Method 1 : without Data Augmentation**

Steps :
* use the pre-trained model to extract the features
* resize the output so that it can used by the classifier
* train the new classifier
* evaluate. 


## Extract the features with the convolutional base

We use the class **ImageDataGenerator()**. 

In [10]:
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

''' Function 'extract_feature()'
* inputs : 
  * 'directory' : folder containing the images (train_dir, validation_dir or test_dir)
  * 'sample_count' : number of images in the directory (2000 or 1000)
* outputs :
  * 'features' : features extracted, each with the format (4, 4, 512) 
  * 'labels' : labels of the images. 
'''

def extract_feature(directory, sample_count):
    # initialize the arrays 'features' and 'labels' with zeros
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count,))
    # apply the generator to the directory 
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150,150),
        batch_size=batch_size,
        class_mode='binary'
    )
    # use the convolutional base on the generated images (.predict())
    # add the extracted features and the generated labels to the arrays 'features' et 'labels'
    # stop when all the images in the directory are treated
    i=0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i*batch_size : (i+1)*batch_size] = features_batch 
        labels[i*batch_size : (i+1)*batch_size] = labels_batch
        i += 1
        if i*batch_size >= sample_count:
            break
    return features, labels

# apply the function 'extract_feature()' to the 3 directories train_dir, validation_dir, test_dir
train_features, train_labels = extract_feature(train_dir, 2000)
validation_features, validation_labels = extract_feature(validation_dir, 1000)
test_features, test_labels = extract_feature(test_dir, 1000)

## Resize the extracted features

The extracted features have the format (samples, 4, 4, 512) : we flatten them into the format (samples, 4x4x512).

In [11]:
train_features = np.reshape(train_features, (2000, 4*4*512))
validation_features = np.reshape(validation_features, (1000, 4*4*512))
test_features = np.reshape(test_features, (1000, 4*4*512))

## Define the new classifier

In [12]:
model = models.Sequential()

model.add(layers.Dense(256, activation='relu', input_dim=4*4*512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',  # optimizers.RMSprop(lr=1e-4) ??
             loss='binary_crossentropy',
             metrics=['accuracy'])

## Train the classifier

In [13]:
history = model.fit(
            train_features, train_labels,
            epochs=30, 
            batch_size=20,
            validation_data=(validation_features, validation_labels)
)

In [14]:
# save the trained model
model.save('classifying_cats_dogs_small_3.h5')

## Evaluate the model

In [15]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'bo', label='training')
plt.plot(epochs, val_acc, 'b', label='validation')
plt.title("Training and Validation Accuracy")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='training')
plt.plot(epochs, val_loss, 'b', label='validation')
plt.title("Training and Validation Loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

The accuracy is pretty good, around 90%, but the model is in overfitting from the beginning.

***

# **Method 2 : with Data Augmentation**

Steps :
* generate the images with data augmentation
* build the model with the convolution base and the new classifier together 
* freeze the convolutional base, don't need to be trained
* train the model
* evaluate.

## Generate the images with data augmentation

In [16]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2, 
                                  horizontal_flip=True, 
                                  fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
                        train_dir,
                        target_size=(150,150),
                        batch_size=20,
                        class_mode='binary' 
                        )

validation_generator = test_datagen.flow_from_directory(
                        validation_dir,
                        target_size=(150,150),
                        batch_size=20,
                        class_mode='binary' 
                        )

# Build the model

In [17]:
model = models.Sequential()

model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
#model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

In [18]:
model.summary()

## Freeze the convolutional base

In [19]:
print("Number of trainable weights before freezing:", len(model.trainable_weights))
conv_base.trainable = False
print("Number of trainable weights after freezing:", len(model.trainable_weights))

## Train the model

In [20]:
model.compile(optimizer='rmsprop',  # optimizers.RMSprop(lr=1e-4) ??
             loss='binary_crossentropy',
             metrics=['accuracy'])

history = model.fit_generator(
            train_generator,
            #steps_per_epoch=100,     
            epochs=30, 
            validation_data=validation_generator,
            #validation_steps=50      
)

In [21]:
# save the trained model 
model.save('classifying_cats_dogs_small_4.h5')

# Evaluate the model

In [22]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'bo', label='training')
plt.plot(epochs, val_acc, 'b', label='validation')
plt.title("Training and Validation Accuracy")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='training')
plt.plot(epochs, val_loss, 'b', label='validation')
plt.title("Training and Validation Loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

###Accuracy is 90.8% , Performing binary classification where 0 stands for CAT and 1 stands for DOG

In [32]:
# make a prediction for a new image.
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model

# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, target_size=(150, 150))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 150, 150, 3)
	# center pixel data
	img = img.astype('float32')
	#img = img - [123.68, 116.779, 103.939]
	return img

# load an image and predict the class
def run_example():
	# load the image
	img = load_image('../input/microsoft-catsvsdogs-dataset/PetImages/Cat/1000.jpg')
	# load model
	model = load_model('classifying_cats_dogs_small_4.h5')
	# predict the class
	result = model.predict(img)
	print(result[0])

# entry point, run the example
run_example()